# <span style="color:green"><center>Diplomado en Big Data</center></span>

# <span style="color:red"><center>HDF5<center></span>

##   <span style="color:blue">Profesores</span>

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co
1. Daniel Mauricio Montenegro Reyes, dextronomo@gmail.com 


##   <span style="color:blue">Asesora Medios y Marketing digital</span>
 

1. Maria del Pilar Montenegro, pmontenegro88@gmail.com 

## <span style="color:blue">Asistentes</span>

1. Oleg Jarma, ojarmam@unal.edu.co 

## <span style="color:blue">Contenido</span>

* [Introducción](#Introducción)
* [Primeros pasos con archivos HDF5](#Primeros-pasos-con-archivos-HDF5)
* [Datasets HDF5](#Datasets-HDF5)
* [Fragmentación (chunking) y compresión](#Fragmentación-(chunking)-y-compresión)
* [Grupos, enlaces e iteración](#Grupos,-enlaces-e-iteración)


## <span style="color:blue">Introducción</span>

En esta lección revisamos uno de lso formatos más usados para el almacenamiento de datos de tipo científicos. HDF5 es recomendado para registrar informaciones secuenciales, por ejemplo proveniente de medidas de equipos científicos, salud, etc.

Los datos que se almacenan en archivos de tipo HDF5 se llaman datasets y básicamente corresponden a arrays multidimensionales.

Cuando los datos son de tipo tabular (tipos tabla de datos o dataframes) otrso formatos son recomendados, como por ejemplo el formato *parquet*.

In [1]:
#! conda install h5py

### Importar modulos

In [2]:
import numpy as np
import h5py

import os

In [3]:
### define carpeta de trabajo

### Organización de datos y metadatos

### Genera datos simulados

In [4]:
# reproducibilidad
np.random.seed(100)

# Temperatura
temperatura_15 = np.random.random(1024) # temperaturas estación. 15
temperatura_10 = np.random.random(1024)

# Viento
viento_15 = np.random.random(2048)
viento_10 = np.random.random(2048)

estacion_15 = 15  # estación 15
estacion_10 = 10

dt_temp = 10 # delta-T temperatura
dt_viento = 20

start_time_temp_15 = 1375204299 # en Unix time
start_time_temp_10 = 1375204500 # en Unix time
start_time_viento_15 = 137550000
start_time_viento_10 = 137550000



In [5]:
print(temperatura_15)

[0.54340494 0.27836939 0.42451759 ... 0.24908888 0.27119505 0.05880634]


### Vamos a crear la siguiente estructura

* Archivo : clima.hdf5
* Estación (15 para el ejemplo): /15/
* Temperatura: /15/temperatura
* Viento: /15/viento

In [6]:
folder = '../Datos/'
file_name = "clima.hdf5" # archivo
file = os.path.join(folder, file_name)

f = h5py.File(file, mode='w') 
print(file)
print(f)

../Datos/clima.hdf5
<HDF5 file "clima.hdf5" (mode r+)>


In [7]:
 
f['/15/temperatura'] = temperatura_15
f['/15/temperatura'].attrs["dt"] = dt_temp
f['/15/temperatura'].attrs['star_time'] = start_time_temp_15

f['/15/viento'] = viento_15
f['/15/viento'].attrs["dt"] = dt_viento
f['/15/viento'].attrs['star_time'] = start_time_viento_15

f['/10/temperatura'] = temperatura_10
f['/10/temperatura'].attrs["dt"] = dt_temp
f['/10/temperatura'].attrs['star_time'] = start_time_temp_10

f['/10/viento'] = viento_10
f['/10/viento'].attrs["dt"] = dt_viento
f['/10/viento'].attrs['star_time'] = start_time_viento_10

f.close()

### Recupera datos del archivo

In [8]:
file = os.path.join(folder, file_name)
f = h5py.File(file, mode='r') 


In [9]:
f.keys()

<KeysViewHDF5 ['10', '15']>

In [10]:
group_15 = f['/15']

In [11]:
group_15

<HDF5 group "/15" (2 members)>

In [12]:
group_15.keys()

<KeysViewHDF5 ['temperatura', 'viento']>

In [13]:
dataset = f['/15/temperatura']

for key, value in dataset.attrs.items():
    print('%s: %s' %  (key, value))

dt: 10
star_time: 1375204299


### Rebanado(slicing) de datasets

In [14]:
dataset[0:10]

array([0.54340494, 0.27836939, 0.42451759, 0.84477613, 0.00471886,
       0.12156912, 0.67074908, 0.82585276, 0.13670659, 0.57509333])

In [15]:
type(dataset)

h5py._hl.dataset.Dataset

In [16]:
np.mean(dataset)

0.4948712717532915

In [17]:
dataset.shape

(1024,)

## <span style="color:blue">Primeros pasos con archivos HDF5</span>

HDF5 utiliza un sistema de tipos muy similar a Numpy. Cada `array` (dataset) o conjunto de datos en un archivo HDF5 tiene un tipo fijo representado por un objeto de tipo. El paquete `h5py` mapea automáticamente el HDF5
type system en NumPy dtypes, lo que, entre otras cosas, facilita el intercambio
datos con NumPy.

Por ejemplo, HDF5, toma prestada esta sintaxis de "corte" para permitir cargar solo porciones de un conjunto de datos

### Mi primer archivo hdf5 

In [ ]:
import h5py

f = h5py.File("../Datos/nombre.hdf5","a")
f.close()

### Modos de abrir una archivo

### Usando un manejador de contexto

El  ejemplo típico de manejo de contexto con archivos es

In [ ]:
with open("../data/garbage.txt","w") as f:
    f.write("Hello!")
    

Cuando se sale del contexto, el archvo se cierra automáticamente.

Podemos hacer lo mismo con archivos hdf5

In [ ]:
with h5py.File("../data/nombre.hdf5","w") as f:
    print(f["dataset_perdido"]) # error dataset no existe en el archivo



In [ ]:
list(f.keys()) # error. Archivo cerrado

In [ ]:
print(f)

In [ ]:
type(f)

### Controladores de archivos (file drivers)

Los controladores de archivos se encuentran entre el sistema de archivos y el mundo de alto nivel de los grupos HDF5, los datasets y atributos. Los controladores se ocupan de la mecánica del *manejo de espacio en memoria* de los   archivos HDF5 disponibles en el disco.

Por lo general, no tendrá que preocuparse por cúal controlador está en uso, ya que el controlador predeterminado funciona bien para la mayoría de las aplicaciones.

Lo mejor de los controladores es que una vez que se abre el archivo, son totalmente transparentes.
Simplemente use la biblioteca HDF5 como de costumbre, y el controlador se encarga de la mecánica de almacenamiento.

#### Controlador (driver) core

El controlador `core` almacena su archivo completamente en la memoria. Obviamente, hay un límite en cuanto a cómo
gran cantidad de datos que puede almacenar, pero la compensación es lecturas y escrituras increíblemente rápidas. Es una gran elección cuando desea la velocidad de acceso a la memoria, pero también desea utilizar el HDF5
estructuras. 

In [ ]:
f = h5py.File('../data/nombre.hdf5', driver='core')

Para decirle a HDF5 que cree un archivo  y que guarde la imagen actual del archivo cuando se cierre, puede usar `backing store`:

In [ ]:
f = h5py.File('../data/nombre.hdf5', driver='core', backing_store=True)

#### Controlador family

Permite separar un archivo en múltiple imágenes, cada de la cuales comparte cierto tamaño máximo. 

In [ ]:
# divide el archivo en trozos de 1Gb de memoria
f = h5py.File('../data/family.hdf5', memb_size=1024**3)

El tamaño por defecto es memb_size = $2^{31}-1$.

#### Controlador mpio

Este controlador es el corazón de Parallel HDF5. Le permite acceder al mismo archivo desde múltiples
procesos al mismo tiempo. Puede tener docenas o incluso cientos de procesos de computación paralela, todos los cuales comparten una vista coherente de un solo archivo en el disco.

### El bloque del usuario (user block)

Una característica interesante de HDF5 es que los archivos pueden estar precedidos por datos arbitrarios de usuario.

Cuando se abre un archivo, la biblioteca busca el encabezado HDF5 al comienzo del
archivo, luego 512 bytes en, luego 1024, y así sucesivamente en potencias de 2. 

Dicho espacio al principio del archivo se denomina `bloque de usuario` y allí el usuario puede almacenar los datos que desee.
Las únicas restricciones están en el tamaño del bloque (potencias de 2 y al menos 512), y que
no debería tener el archivo abierto en HDF5 al escribir en el bloque de usuario. Aquí hay un
ejemplo:

In [20]:
f = h5py.File('../Datos/userblock_example.hdf5', 'w', userblock_size=512)
print(f.userblock_size)
f.close()

512


In [23]:
palabra = 'a'*512
with open('../Datos/userblock_example.hdf5', 'r+') as f:
    f.write(palabra)

In [26]:
with open('../Datos/userblock_example.hdf5', 'rb+') as f: # abre como binario porque así está almacenado
    pal = f.read(512)
pal

b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'

## <span style="color:blue">Datasets HDF5</span>

Los conjuntos de datos (`datasets`) son la característica central de HDF5. Puede pensar en ellos como arreglos (arrays) NumPy que viven en disco. Cada conjunto de datos en HDF5 tiene un nombre, un tipo y una forma, y admite datos aleatorios.
acceso. 

A diferencia de np.save y friends integrados en Numpy, no es necesario leer ni escribir el arreglo  completo como un bloque; puede usar la sintaxis estándar de NumPy para cortar, leer o
escribir solo las partes que desee del array.

### Elemento escenciales sobre datasets

Primero, creemos un archivo para que tengamos un lugar donde almacenar nuestros conjuntos de datos:

In [27]:
f = h5py.File('../Datos/testfile.hdf5', 'w')

Cada conjunto de datos de un archivo HDF5 tiene un nombre. Veamos qué sucede si asignamos un nuevo
array NumPy a un nombre en el archivo:

In [28]:
import numpy as np

arr = np.ones((5,2))
f['my_dataset'] = arr
dset = f['my_dataset'] 
dset

<HDF5 dataset "my_dataset": shape (5, 2), type "<f8">

Observe que *dset* es una instancia de la clase *h5py-Dataset*. Este objeto se accesa como cualquier archivo Numpy. 

In [29]:
print(dset[:])
print(dset[...])

[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]


### Type y shape

In [30]:
print(dset.dtype)
print(dset.shape)
print(dset[:, -1])
out = dset[...]
print(type(out))
print(out.shape)

float64
(5, 2)
[1. 1. 1. 1. 1.]
<class 'numpy.ndarray'>
(5, 2)


### Creación de datasets vacíos

No es necesario tener una matriz NumPy lista para crear un conjunto de datos.

In [31]:
dset = f.create_dataset('test1', (10,10))


In [32]:
f.keys()

<KeysViewHDF5 ['my_dataset', 'test1']>

In [33]:
dset[...]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

HDF5 es lo suficientemente inteligente como para asignar solo la cantidad de espacio en el disco que realmente necesita
almacenar los datos que escribe. A continuación, se muestra un ejemplo: suponga que desea crear un conjunto de datos 1D
que puede contener muestras de datos de 4 gigabytes de un experimento de larga duración:

In [ ]:
dset = f.create_dataset('big_dataset', (1024**3), dtype= np.float32)
dset[0:1024] = np.arange(1024)
f.flush() # envia los datos al archivo
#f.close()

# Por favor revise el tamaño del archivo en el sistema operativo

In [ ]:
# ls -lh testfile.hdf5

### Conversión de tipos al enviar al archivo hdf5

In [ ]:
import numpy as np
import h5py

bigdata = np.ones((100,1000)) # np.float64
print(bigdata.dtype)
print(bigdata.shape)

Guarda con el tipo de dato original: np.float64

In [ ]:
with h5py.File('../Datos/big1.hdf5','w') as f1:
    f1['big'] = bigdata

Cambia el tipo de dato al enviar al archivo

In [ ]:
with h5py.File('../Datos/big2.hdf5', 'w') as f2:
    f2.create_dataset('big', data=bigdata, dtype=np.float32)

In [ ]:
Recupera los datos

In [ ]:
f1 = h5py.File('../Datos/big1.hdf5')
f2 = h5py.File('../Datos/big2.hdf5')
print(f1['big'].dtype)
print(f2['big'].dtype)

Conversión automática de tipos

La propia biblioteca HDF5 maneja la conversión de tipos y lo hace sobre la marcha (*on the fly*) cuando se guarda en o se lee de un archivo.

Vemos como sucede esto.

In [ ]:
dset = f2['big']
print(dset.dtype)
print(dset.shape)

En este momento *dset* apunta a los datos en el archivo, pero no han sido cargados en memoria. Para cargarlos con 'float64' lo que debe hacer asignar primero un arreglo vacío de doble precisión en la memoria:

In [ ]:
big_out = np.empty((100, 1000), dtype= np.float64)

Ahora cargamos los datos a la memoria en ese espacio asignao en memoria. La API de HDF5 hace la conversón por el camino (*on the fly*):

In [ ]:
dset.read_direct(big_out)
print(dset.dtype)
print(big_out.dtype)

*dset* sigue apuntando al archivo, mientras que big_out apunta a los datos (convertidos) en memoria.

En realidad con *read_direct* no necesariamente tiene que leer todos los datos.

#### Lectura con astype

In [ ]:
with dset.astype('float64'):
    out = dset[0,:]

print(out.dtype)
print(out.shape)

### Lectura y escritura de datos

En esta seccion estudiamos inportantes asuntos de implementación  que mejoran el desempeño de los programas con hdf5. Primero estudiamos algunas diferencias entre los arreglos de Numpy y los datasets de hdf5.

#### Rebanado

Empezamos leyendo de un dataset existente

In [ ]:
import numpy as np
import h5py

f2 = h5py.File('../Datos/big1.hdf5','r+')
print(f2.keys())

In [ ]:
dset = f2['big']
dset

In [ ]:
out = dset[0:10, 20:70]
out.shape

#### ¿Que ocurrió?

1. h5py calcula la forma (10, 50) del array resultante.
2. A un array  NumPy vacío se le asigna la forma (10, 50).
3. HDF5 selecciona la parte apropiada del conjunto de datos.
4. HDF5 copia datos del conjunto de datos en el arreglo NumPy vacía.
5. Se devuelve el arreglo NumPy recién llenado.

#### Tip 1

Tomar rebabadas de tamaño razonable. Revise los dos siguientes snippets de código. ¿Cuál es más eficiente?

In [ ]:
# Chequear valiores negativos y reemplazarlos por cero (0).
from time import time

In [ ]:
%time
f2 = h5py.File('../Datos/big1.hdf5','r+')

for ix in range(100):
    for iy in range(1000):
        val = dset[ix,iy]     # lee elemento
        if val < 0: 
            dset[ix,iy] = 0   # recorta a 0 si es negativo
    

In [ ]:
# Chequear valiores negativos y reemplazarlos por cero (0).

%time
for ix in range(100):
    val = dset[ix,:] # lee una fila
    val[val<0] = 0    # recorta si es negativo
    dset[ix,:] = val  # escribe de regreso en el dataset hdf5
    

#### Comparación con dask- array

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

In [ ]:
import dask.array as da
x = da.from_array(dset, chunks=(100))
x

In [ ]:
%time

x[x<0] = 0
dset[:] = x

In [ ]:
f2.close()

#### Indexación de datasets hdf5

In [ ]:
import numpy as np
import h5py


f = h5py.File('../Datos/example1.hdf5','w')
dset = f.create_dataset('range', data=range(10))

In [ ]:
print(dset[4])   # un elemento
print(dset[4:8]) # rebanado
print(dset[...]) # notación puntos suspensivos (ellipsis) todo
print(dset[:])   # todo en una dimensión
print(dset[4:-1]) # notación del último elemento
print(dset[::-1])   # Fallo. Recorrer en modo reverse

#### Rebanado multidimensional

In [ ]:
dset = f.create_dataset('4d', shape=(100,80,50,20))
print(dset.shape)
print(dset[...].shape)

In [ ]:
dset = f.create_dataset('1d', shape=(1,), data = 42)

print(dset[0])  
print(dset.shape)
print(dset[...].shape) 
print(dset[:])  

#### Indexación boolena

Como en Numpy. se pueden usar máscara con valores boolenos para extraer  elementos de un dataset.

In [ ]:
data = np.random.random(10)*2 -1
data

In [ ]:
dset = f.create_dataset('random', data=data)
dset[data<0] = 0
dset[...]

In [ ]:
dset[data<0] = -1*data[data<0]
dset[...]

#### Coordenadas con listas

In [ ]:
dset[[1,2,7]]

#### Leyendo directamente en un arreglo existente

Al leer directamente en un array existente se llena el arrelgo y se hace el recast (conversión de tipo) de manera automática

In [ ]:
dset = f.create_dataset('100_1000_array', shape=(100, 1000), dtype=np.float32)

In [ ]:
dset

In [ ]:
out = np.empty((100,1000), dtype=np.float64)
dset.read_direct(out)
out

Supongamos que queremos leer la primera fila, en dset [0 ,:], y depositarlo
en el array de salida en fila 50: out[50 ,:]. Podemos utilizar las palabras clave *source_sel* y *dest_sel*, para la selección de la fuente y la selección del destino respectivamente:

In [ ]:
dset.read_direct(out, source_sel = np.s_[0,:], dest_sel= np.s_[50,:])

Observe los dos siguientes snippets de código. ¿Cuál es más eficiente?

In [ ]:
out = dset[:, 0:50]
print(out.shape)
means = out.mean(axis=1)
print(means.shape)

In [ ]:
out = np.empty((100,50),dtype = np.float32)
dset.read_direct(out, np.s_[:,0:50])
mean = out.mean(axis=1)

Este puede parecer un caso trivial, pero hay una diferencia importante entre los dos  enfoques. 

* En el primer ejemplo, h5py crea internamente la matriz out, que se utiliza para almacenar la rebanada y luego desecharla. 
* En el segundo ejemplo, out es asignado por el usuario y se puede reutilizar para futuras llamadas a read_direct.

No hay dierencia en el desempeño, pero ahora evaluamos un arreglo grande

In [ ]:
dset = f.create_dataset('test', shape=(10000, 10000), dtype=np.float32)

In [ ]:
dset[:] = np.random.random(10000) # uso de broadcasting para completar

In [ ]:
from time import time
from dask.distributed import Client
client = Client(n_workers=4)
import dask.array as da



In [ ]:
%time
dset[:,0:500].mean(axis=1)

In [ ]:
#numpy
out = np.empty((10000, 500), dtype=np.float32)
%time
dset.read_direct(out, np.s_[:,0:500])
out.mean(axis=1)

In [ ]:
# dask
%time
x = da.from_array(dset[:,0:500], chunks=(4))
y= x.mean(axis=1)
y.compute()

In [ ]:
def time_dataset():
    dset[:,0:500].mean(axis=1)

def time_numpy():
    dset.read_direct(out, np.s_[:,0:500])
    out.mean(axis=1)

#def time_dask():
#    x = da.from_array(dset[:,0:500], chunks=(4))
#    y = x.mean(axis=1).compute()

In [ ]:
from timeit import timeit

In [ ]:
timeit(time_dataset, number=100)


In [ ]:
timeit(time_numpy, number=100)

In [ ]:
#timeit(time_dask, number=100)

### Cambio de tamaño de los datasets

Así como en Numpy se puede cambiar el tamaño de los arreglos, mientras se mantenga la coherencia en el tamaño global, dejando al usuario la responsabilidad de la interpretabilidad de los datos, con los datasets de hdf5 podemos hacer lo mismo, pero es posible perder información. Veámos.


In [ ]:
dset = f.create_dataset('fixed', (2,2))
print(dset.shape)
print(dset.maxshape)

La propiedad maxshape sugiere que el tamaño podría variar hasta esos extremos. En efecto, asi es. Veámos.

In [ ]:
dset = f.create_dataset('resizable', (2,2), maxshape=(2,2))
print(dset.shape)
print(dset.maxshape)

In [ ]:
dset.resize((1,1))
print(dset.shape)
print(dset.maxshape)

In [ ]:
dset.resize((1,3)) # falla porque 3>2
print(dset.shape)
print(dset.maxshape)

No esposible cambiar las dimensiones, es decir, el número total de ejes, de una dataset. Pero si se pueden dejar dimensiones sin un tamaño definido para que crezcan idefinidamente. Basta colocar en la posición respectiva *None* cuando crea el dataset. Veámos

In [ ]:
dset = f.create_dataset('unlimited', (2,2), maxshape =(2,None))
print(dset.shape)
print(dset.maxshape)

In [ ]:
dset.resize((2,2*30))
print(dset.shape)

#### Cuidados con el cambio de tamaños en datasets

Las reglas de Numpy no aplican exactamento a los dataset. Observe el siguiente ejemplo Numpy

In [ ]:
a = np.array([[1,2],[3,4]])
print(a.shape)
print(a)

a.resize((1,4))
print(a)

a.resize((1,10))
print(a)        

Ahora veámos el mismo código con dataset

In [ ]:
dset = f.create_dataset('sizetest', (2,2), dtype=np.int32, 
                        maxshape=(None, None))

In [ ]:
dset[...] = [[1,2],[3,4]]
print(dset)

In [ ]:
dset[...]

In [ ]:
dset.resize((1,4))
dset[...] # se perdió la fila 1, hdf5 no pudo hacer el reordenamiento

In [ ]:
dset.resize((1,10))
dset[...]

#### Cuando y como hacer el cambio de tamaño del dataset

Mostramos dos vías. La segunda es más recomendada.

1. Se van agregando elementos a la medida que el datset crece
2. Se crea un dataset grande y luego se poda

In [ ]:
# Agregado al paso

dset1= f.create_dataset('time_traces', (1,1000), maxshape=(None,1000))

def add_trace(arr):
    dset.resize(dset1.shape[0]+1, 1000)
    dset[-1,:] = arr

In [ ]:
# creando dataset grande y al final podando

dset2 = f.create_dataset('time_traces_2', (5000,1000), maxshape=(None,1000))

ntraces= 0
def add_trace_2(arr):
    global ntraces
    dset2[ntraces,:] = arr
    ntraces += 1

def done():
    dset.resize((ntraces,1000))
    

## <span style="color:blue">Fragmentación (chunking) y compresión</span> 

Consideremos el siguiente arreglo 2-dimensional de Numpy

In [ ]:
import numpy as np

a = np.array([['A','B'],['C','D']])
print(a)

En realidad el arreglo esta organizado en la memoria linealmente, en el siguiente orden

+ 'A', 'B', 'C', 'D'

Lo mismo ocurre con los datasets de hdf5. Esto implica que algunas forma de recuperacion de información son mpás eficientes que otras.

Suponga que va a almacenar 10 imágenes en b/n (escala de grises) de tamaño 480*640. Entonces haría algo como lo siguiente

In [ ]:
import numpy as np
import h5py

f = h5py.File('imagetest.hdf5','w')
dset = f.create_dataset('Imagenes', (100,480,640))

La primera imagen se recupera como

In [ ]:
imagen = dset[0,:,:]
image.shape  # (480, 640)

Esta imagen  muesra como es almacenado el dataset

![dataset_almacenamiento](../Imagenes/dataset_almacenamiento.jpeg)

## Almacenamiento fragmentado

Pero, ¿qué pasa si, en lugar de procesar imágenes completas una tras otra, nuestra aplicación trata con mosaicos de imagen? 

Supongamos que queremos leer y procesar los datos en un segmento de 64 × 64 píxeles en la esquina de la primera imagen; por ejemplo, digamos que queremos agregar un logotipo.
Nuestra selección de rebanado sería:

In [ ]:
tile = dset[0,0:64, 0.64]
tile.shape # (64,64)

¿Y si hubiera alguna forma de expresar esto de antemano? ¿No hay forma de preservar la forma del conjunto de datos, que es semánticamente importante, pero dígale a HDF5 que optimice la
conjunto de datos para el acceso en bloques de 64 × 64 píxeles?


Eso es lo que hace la fragmentación en HDF5. Le permite especificar la "forma" N-dimensional que
se adapta mejor a su patrón de acceso. Cuando llega el momento de escribir datos en el disco, HDF5 se divide
los datos en "trozos" de la forma especificada, los aplana y los escribe en el disco. Los fragmentos se almacenan en varios lugares del archivo y sus coordenadas están indexadas por un
Árbol B (binario).

Aquí tenemos un ejemplo. Tomemos el conjunto de datos de forma (100, 480, 640) que se acaba de mostrar y digamos a HDF5
que lo almacene en formato fragmentado. 

Hacemos esto proporcionando una nueva palabra clave, fragmentos, al método *create_dataset*:

In [ ]:
dset = f.create_dataset('fragmentado', (100,480,640), dtype='i1',
                       chunks=(1,64,64))

Así son almacenados los datos en este caso.

![fragmentado](../Imagenes/chunckshdf5.jpeg)

In [ ]:
Así es como ocurre la compresión en hdf5.

#### Auto fragmentado

Si no esta seguro sobre como gestionará el dataset, puede dejar que hdf5 decida como fragmentarlo

In [ ]:
dset = f.create_dataset('imagenes2', (100,480,640), chunks=True)
dset.chunks # (13, 60,80)

### Elegir una forma manualmente


A continuación, se incluyen algunas cosas que debe tener en cuenta al trabajar con fragmentos. El proceso de elegir
la forma de los fragmentos es una compensación entre las siguientes tres restricciones:

1. Los fragmentos más grandes para un tamaño de conjunto de datos determinado reducen el tamaño del fragmento del árbol B, lo que hace es más rápido para buscar y cargar fragmentos.
2. Dado que los fragmentos son todo o nada (leer una parte carga todo el fragmento), los fragmentos también aumentan la posibilidad de que lea datos en la memoria que no usará.
3. La caché de fragmentos HDF5 solo puede contener un número finito de fragmentos. Trozos más grandes
de 1 MB ni siquiera compartiran en el caché.

Entonces, estos son los puntos principales a tener en cuenta:

- *¿Necesita realmente especificar un tamaño de fragmento?* Es mejor restringir el tamaño manual de los fragmentos a los casos en los que esté seguro de que su conjunto de datos
se accederá de una manera que probablemente sea ineficaz con el almacenamiento contiguo o una forma de trozo adivinada automáticamente. Y como todas las optimizaciones, ¡debería compararlas!

- *Intente expresar el patrón de acceso "natural" que tendrá su conjunto de datos*. Como en nuestro ejemplo, si está almacenando un montón de imágenes en un conjunto de datos y sabe que
su aplicación leerá "mosaicos" particulares de 64 × 64, podría usar N × 64 × 64 trozos (o N × 128 × 128) a lo largo de los ejes de la imagen.

- *No los hagas demasiado pequeños*. Tenga en cuenta que HDF5 tiene que usar datos de indexación para realizar un seguimiento de las cosas; si utiliza algo patológico como un tamaño de fragmento de 1 byte, la mayor parte de su espacio en disco será
tomado por metadatos. Una buena regla general para la mayoría de los conjuntos de datos es mantener fragmentos
por encima de 10 KB más o menos.

- *No los hagas demasiado grandes*. La clave para recordar es que cuando lee cualquier dato en un fragmento, todo el
se lee el fragmento. Si solo usa un subconjunto de los datos, el tiempo extra dedicado a la lectura de el disco está desperdiciado. Tenga en cuenta que los trozos de más de 1 MiB de forma predeterminada no participar en el rápido "caché de fragmentos" en memoria y, en su lugar, se leerá desde el disco
cada vez.

### Filtros y Compresión

Con la fragmentación, es posible realizar la compresión de forma transparente en un conjunto de datos.

Se conoce el tamaño inicial de cada fragmento y, dado que están indexados por un árbol B, pueden
almacenarse en cualquier lugar del archivo, no solo uno tras otro. En otras palabras, cada trozo es
libre para crecer o encogerse sin golpear a los demás

####  Tubería de filtro

HDF5 tiene el concepto de una tubería de filtro, que es solo una serie de operaciones realizadas
en cada fragmento cuando está escrito. Cada filtro es libre de hacer lo que quiera con los datos en
el fragmento: comprimirlo, sumarlo, agregar metadatos, cualquier cosa. Cuando se lee el archivo, cada
El filtro se ejecuta en modo "inverso" para reconstruir los datos originales.

La imagen muestra como sucede la acción


![Tuberia_filtro](../Imagenes/tuberia_filtros.jpeg)

#### Filtros de compresión

Hay varios filtros de compresión disponibles en HDF5. Con mucho, el más utilizado es el filtro GZIP. (También escuchará que se hace referencia a esto como el filtro "DEFLATED"; en el mundo HDF5, ambos nombres se utilizan para el mismo filtro).

A continuación, se muestra un ejemplo de compresión GZIP utilizada en un conjunto de datos de punto flotante:

In [ ]:
dset =  f.create_dataset('BigDataset', (1000,1000),dtype='f', 
                         compression='gzip' )
dset.compression # 'gzip

## <span style="color:blue">Grupos, enlaces e iteración</span> 

Los grupos son el objeto contenedor HDF5, análogo a las carpetas de un sistema de archivos. Ellos pueden
contener conjuntos de datos y otros grupos, lo que le permite construir una estructura jerárquica con
objetos perfectamente organizados en grupos y subgrupos

#### El grupo raiz y subgrupos

El objeto de grupo más general es h5py.Group, del cual h5py.File es una subclase. Otro Los grupos se crean fácilmente con el método *create_group*:

In [ ]:
#import h5py
f = h5py.File('../Datos/Groups.hdf5','w')
subgroup = f.create_group('SubGroup')
subgroup

In [ ]:
subgroup.name

Por supuesto, los grupos también se pueden anidar. El método *create_group* existe en todos los grupos objetos, no solo File:

In [ ]:
subsubgroup = subgroup.create_group('AnotherGroup')
subsubgroup.name

No es necesario crear manualmente todos los subgrupos anidados.

In [ ]:
out = f.create_group('/some/big/path')
out

In [ ]:
f.keys()

### Elementos escenciales de Group

Si no recuerda nada más de este capítulo, recuerde esto: los grupos funcionan principalmente como diccionarios. 

Hay un par de agujeros en esta abstracción, pero en general funciona sorprendentemente bien. Los grupos son iterables y tienen un subconjunto del diccionario Python normal
API.

Agregemos unos pocos objetos al archivo de este ejemplo para lo que sigue.

In [ ]:
f['Dataset1'] = 1.0
f['Dataset2'] = 2.0
f['Dataset3'] = 3.0
subgroup['Dataset4'] = 4.0


#### Acceso al estilo de diccionario

In [ ]:
# accesa el dataset asociado a la 'clave' dataset1
dset1 = f['Dataset1']

dset4 = f['SubGroup/Dataset4']


In [ ]:
print(len(f))
print(len(f['SubGroup']))